In [1]:
import pandas as pd
import numpy as np
import os
from datetime import date, datetime
from dateutil.relativedelta import relativedelta

In [2]:
base_path = './data/'

customer_filename = 'customers.csv'
product_filename = 'product_usage.tsv'
subscription_filename = 'subscriptions.json'
support_filename = 'support_tickets.csv'

In [3]:
customer_df = pd.read_csv(os.path.join(base_path, customer_filename))
product_df = pd.read_csv(os.path.join(base_path, product_filename), sep='\t')
subscription_df = pd.read_json(os.path.join(base_path, subscription_filename))
support_ticket_df = pd.read_csv(os.path.join(base_path, support_filename))

In [4]:
# keeping copies of original data for fail-safe conditions
customer_df_original = customer_df.copy()
product_df_original = product_df.copy()
subscription_df_original = subscription_df.copy()
support_ticket_df_original = support_ticket_df.copy()

In [5]:
customer_df.head(3)

,customer_id,name,signup_date,industry,plan,location
0,1,Customer_1,2023-10-17,Fintech,Basic,NaN
1,2,Customer_2,2022-10-09,E-commerce,Pro,San Francisco
2,3,Customer_3,2022-04-15,NaN,Basic,London


In [6]:
product_df.head(3)

,customer_id,usage_date,features_used,minutes_spent,errors_reported
0,34,2024/03/31,Upload,59.90,1
1,97,2024/04/25,Reports,40.85,0
2,190,2024/05/16,Export,53.37,1


In [7]:
subscription_df.head(3)

,customer_id,subscription_id,start_date,end_date,status,payment_method
0,1,sub_1,28-06-2023,20-01-2024,active,upi
1,2,sub_2,27-12-2022,15-09-2023,active,upi
2,3,sub_3,25-08-2022,24-11-2022,inactive,upi


In [8]:
support_ticket_df.head(3)

,ticket_id,customer_id,issue_type,opened_date,resolved_date,satisfaction_score
0,TCKT-1,81,Feature Request,2023-02-03,2023-02-03,4.0
1,TCKT-2,83,Login Issue,2023-02-09,2023-02-11,NaN
2,TCKT-3,177,Bug,2023-10-13,2023-10-24,NaN


##### Let's cleanup one by one

In [9]:
customer_df

,customer_id,name,signup_date,industry,plan,location
0,1,Customer_1,2023-10-17,Fintech,Basic,NaN
1,2,Customer_2,2022-10-09,E-commerce,Pro,San Francisco
2,3,Customer_3,2022-04-15,NaN,Basic,London
3,4,Customer_4,2023-03-09,Fintech,Basic,New York
4,5,Customer_5,2022-08-12,E-commerce,Basic,London
...,...,...,...,...,...,...
195,196,Customer_196,2023-07-23,E-commerce,NaN,NaN
196,197,Customer_197,2023-12-30,Education,NaN,New York
197,198,Customer_198,2022-04-06,Healthcare,Pro,Berlin
198,199,Customer_199,2023-12-10,E-commerce,Enterprise,NaN


In [10]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  200 non-null    int64 
 1   name         185 non-null    object
 2   signup_date  189 non-null    object
 3   industry     162 non-null    object
 4   plan         153 non-null    object
 5   location     165 non-null    object
dtypes: int64(1), object(5)
memory usage: 9.5+ KB


In [11]:
customer_df.isna().sum()

customer_id     0
name           15
signup_date    11
industry       38
plan           47
location       35
dtype: int64

In [12]:
# dropping rows where all the values are blank
customer_df.dropna(how='all')

,customer_id,name,signup_date,industry,plan,location
0,1,Customer_1,2023-10-17,Fintech,Basic,NaN
1,2,Customer_2,2022-10-09,E-commerce,Pro,San Francisco
2,3,Customer_3,2022-04-15,NaN,Basic,London
3,4,Customer_4,2023-03-09,Fintech,Basic,New York
4,5,Customer_5,2022-08-12,E-commerce,Basic,London
...,...,...,...,...,...,...
195,196,Customer_196,2023-07-23,E-commerce,NaN,NaN
196,197,Customer_197,2023-12-30,Education,NaN,New York
197,198,Customer_198,2022-04-06,Healthcare,Pro,Berlin
198,199,Customer_199,2023-12-10,E-commerce,Enterprise,NaN


In [13]:
# checking duplicate values based on PK column, i.e. (customer_id, name)
customer_df[customer_df.duplicated(subset=['customer_id', 'name'])]

,customer_id,name,signup_date,industry,plan,location


In [14]:
# checking for nulls in the customer column
customer_df[customer_df['name'].isna()]

,customer_id,name,signup_date,industry,plan,location
12,13,NaN,2022-03-22,SaaS,Enterprise,NaN
25,26,NaN,2023-11-03,Education,NaN,NaN
38,39,NaN,2022-07-02,SaaS,Basic,NaN
51,52,NaN,2023-08-26,SaaS,Pro,London
64,65,NaN,2022-10-13,Education,Pro,New York
77,78,NaN,2023-09-02,Fintech,Pro,Berlin
90,91,NaN,2023-07-22,Healthcare,Basic,San Francisco
103,104,NaN,2022-02-10,Healthcare,Pro,NaN
116,117,NaN,2022-09-25,E-commerce,Enterprise,New York
129,130,NaN,2023-07-16,Fintech,Enterprise,Bangalore


In [15]:
# replacing the missing customer names using other name patterns available
customer_df.loc[customer_df['name'].isna(), 'name'] = 'Customer_' + customer_df['customer_id'].astype(str)

In [16]:
# conversion to datetime object for future operations
customer_df['signup_date'] = pd.to_datetime(customer_df['signup_date'], errors='coerce', format='%Y-%m-%d')

In [17]:
customer_df['signup_date'].head()

0   2023-10-17
1   2022-10-09
2   2022-04-15
3   2023-03-09
4   2022-08-12
Name: signup_date, dtype: datetime64[ns]

In [18]:
print(customer_df['industry'].unique())
print(customer_df['plan'].unique())
print(customer_df['location'].unique())

['Fintech' 'E-commerce' nan 'SaaS' 'Healthcare' 'Education']
['Basic' 'Pro' nan 'Enterprise']
[nan 'San Francisco' 'London' 'New York' 'Bangalore' 'Berlin']


In [19]:
customer_df['industry'] = customer_df['industry'].str.strip()
customer_df['plan'] = customer_df['plan'].str.strip()
customer_df['location'] = customer_df['location'].str.strip()

In [20]:
customer_df['plan'].fillna('Unknown', inplace=True)
customer_df['industry'].fillna('Unknown', inplace=True)
customer_df['location'].fillna('Unknown', inplace=True)

C:\Users\shrey\AppData\Local\Temp\ipykernel_2568\3135988210.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customer_df['plan'].fillna('Unknown', inplace=True)
C:\Users\shrey\AppData\Local\Temp\ipykernel_2568\3135988210.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

In [21]:
customer_df

,customer_id,name,signup_date,industry,plan,location
0,1,Customer_1,2023-10-17,Fintech,Basic,Unknown
1,2,Customer_2,2022-10-09,E-commerce,Pro,San Francisco
2,3,Customer_3,2022-04-15,Unknown,Basic,London
3,4,Customer_4,2023-03-09,Fintech,Basic,New York
4,5,Customer_5,2022-08-12,E-commerce,Basic,London
...,...,...,...,...,...,...
195,196,Customer_196,2023-07-23,E-commerce,Unknown,Unknown
196,197,Customer_197,2023-12-30,Education,Unknown,New York
197,198,Customer_198,2022-04-06,Healthcare,Pro,Berlin
198,199,Customer_199,2023-12-10,E-commerce,Enterprise,Unknown


#### Products

In [22]:
product_df

,customer_id,usage_date,features_used,minutes_spent,errors_reported
0,34,2024/03/31,Upload,59.90,1
1,97,2024/04/25,Reports,40.85,0
2,190,2024/05/16,Export,53.37,1
3,131,2023/12/03,Dashboard,75.32,2
4,55,2023/11/22,Alerts,31.10,0
...,...,...,...,...,...
4995,19,2024/03/29,API,48.72,0
4996,118,2023/06/21,API,97.74,0
4997,76,2023/11/06,Reports,61.82,1
4998,156,2023/06/01,Reports,3.92,error


In [23]:
product_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   customer_id      5000 non-null   int64  
 1   usage_date       5000 non-null   object 
 2   features_used    5000 non-null   object 
 3   minutes_spent    5000 non-null   float64
 4   errors_reported  5000 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 195.4+ KB


In [24]:
product_df.isna().sum()

customer_id        0
usage_date         0
features_used      0
minutes_spent      0
errors_reported    0
dtype: int64

In [25]:
product_df['usage_date'] = pd.to_datetime(product_df['usage_date'], format='%Y/%m/%d')

In [26]:
product_df['errors_reported'].unique()

array(['1', '0', '2', 'error', '4', '3', '5'], dtype=object)

In [27]:
product_df[product_df['errors_reported'] == 'error']

,customer_id,usage_date,features_used,minutes_spent,errors_reported
6,198,2024-03-03,Upload,6.75,error
33,163,2023-12-16,Reports,43.17,error
45,133,2023-10-15,Dashboard,52.83,error
52,156,2024-05-13,Alerts###,12.82,error
65,52,2023-08-11,API,71.21,error
...,...,...,...,...,...
4984,169,2023-12-04,API,5.67,error
4987,188,2024-04-24,Reports,0.75,error
4990,159,2023-11-09,Alerts,62.08,error
4991,126,2024-02-09,API,46.58,error


In [28]:
product_df['errors_reported'] = pd.to_numeric(product_df['errors_reported'], errors='coerce')

In [29]:
product_df['features_used'].unique()

array(['Upload', 'Reports', 'Export', 'Dashboard', 'Alerts', 'Alerts###',
       'API', 'Dashboard###', 'API###', 'Reports###', 'Upload###',
       'Export###'], dtype=object)

In [30]:
product_df['features_used'] = product_df['features_used'].str.replace(r'[^a-zA-Z ]', '', regex=True).str.strip()

In [31]:
product_df['features_used'].unique()

array(['Upload', 'Reports', 'Export', 'Dashboard', 'Alerts', 'API'],
      dtype=object)

In [32]:
product_df

,customer_id,usage_date,features_used,minutes_spent,errors_reported
0,34,2024-03-31,Upload,59.90,1.0
1,97,2024-04-25,Reports,40.85,0.0
2,190,2024-05-16,Export,53.37,1.0
3,131,2023-12-03,Dashboard,75.32,2.0
4,55,2023-11-22,Alerts,31.10,0.0
...,...,...,...,...,...
4995,19,2024-03-29,API,48.72,0.0
4996,118,2023-06-21,API,97.74,0.0
4997,76,2023-11-06,Reports,61.82,1.0
4998,156,2023-06-01,Reports,3.92,NaN


In [52]:
product_df[~(product_df['customer_id'].isin(customer_df['customer_id']))]

,customer_id,usage_date,features_used,minutes_spent,errors_reported


#### Subscriptions

In [33]:
subscription_df

,customer_id,subscription_id,start_date,end_date,status,payment_method
0,1,sub_1,28-06-2023,20-01-2024,active,upi
1,2,sub_2,27-12-2022,15-09-2023,active,upi
2,3,sub_3,25-08-2022,24-11-2022,inactive,upi
3,4,sub_4,03-05-2022,03-03-2023,cancelled,paypal
4,5,sub_5,16-07-2022,03-12-2022,inactive,upi
...,...,...,...,...,...,...
195,196,sub_196,09-10-2023,04-10-2024,active,upi
196,197,sub_197,07-04-2022,16-02-2023,inactive,upi
197,198,sub_198,16-09-2022,None,inactive,upi
198,199,sub_199,25-03-2022,16-02-2023,inactive,upi


In [34]:
subscription_df.isna().sum()

customer_id         0
subscription_id     0
start_date          0
end_date           18
status              0
payment_method     48
dtype: int64

In [35]:
subscription_df.duplicated().sum()

np.int64(0)

In [36]:
# checking if there's any row hab=ving all blank values
subscription_df[subscription_df.isna().all(axis=1)]

,customer_id,subscription_id,start_date,end_date,status,payment_method


In [37]:
subscription_df['start_date'] = pd.to_datetime(subscription_df['start_date'], format='%d-%m-%Y')
subscription_df['end_date'] = pd.to_datetime(subscription_df['end_date'], format='%d-%m-%Y')

In [38]:
subscription_df['status'].unique()

array(['active', 'inactive', 'cancelled'], dtype=object)

In [39]:
subscription_df['payment_method'].unique()

array(['upi', 'paypal', 'credit_card', None], dtype=object)

In [53]:
subscription_df[~(subscription_df['customer_id'].isin(customer_df['customer_id']))]

,customer_id,subscription_id,start_date,end_date,status,payment_method


#### Support Tickets

In [40]:
support_ticket_df

,ticket_id,customer_id,issue_type,opened_date,resolved_date,satisfaction_score
0,TCKT-1,81,Feature Request,2023-02-03,2023-02-03,4.0
1,TCKT-2,83,Login Issue,2023-02-09,2023-02-11,NaN
2,TCKT-3,177,Bug,2023-10-13,2023-10-24,NaN
3,TCKT-4,169,Feature Request,2023-03-27,2023-04-07,3.0
4,TCKT-5,50,INVALID,2023-05-11,NaN,5.0
...,...,...,...,...,...,...
795,TCKT-796,69,INVALID,2024-04-06,2024-04-19,5.0
796,TCKT-797,21,Login Issue,2023-06-28,2023-07-09,NaN
797,TCKT-798,126,Login Issue,2023-10-09,2023-10-19,5.0
798,TCKT-799,148,Feature Request,2023-09-25,2023-09-27,5.0


In [41]:
support_ticket_df.isna().sum()

ticket_id               0
customer_id             0
issue_type            109
opened_date             0
resolved_date         173
satisfaction_score    175
dtype: int64

In [42]:
support_ticket_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   ticket_id           800 non-null    object 
 1   customer_id         800 non-null    int64  
 2   issue_type          691 non-null    object 
 3   opened_date         800 non-null    object 
 4   resolved_date       627 non-null    object 
 5   satisfaction_score  625 non-null    float64
dtypes: float64(1), int64(1), object(4)
memory usage: 37.6+ KB


In [43]:
support_ticket_df['opened_date'] = pd.to_datetime(support_ticket_df['opened_date'])
support_ticket_df['resolved_date'] = pd.to_datetime(support_ticket_df['resolved_date'])

In [44]:
support_ticket_df['issue_type'].unique()

array(['Feature Request', 'Login Issue', 'Bug', 'INVALID', 'Billing',
       'Other', nan], dtype=object)

In [45]:
support_ticket_df['issue_type'].value_counts(dropna=False)

issue_type
Login Issue        127
Other              126
Feature Request    115
Bug                113
NaN                109
INVALID            108
Billing            102
Name: count, dtype: int64

### Export cleaned dataframes to SQL tables

In [63]:
from sqlalchemy import create_engine

server = 'SHREY-GALAXYBOO'
database = 'churnDB'

connection_string = (
    f"mssql+pyodbc://{server}/{database}"
    "?driver=ODBC+Driver+17+for+SQL+Server"
    "&trusted_connection=yes"
)

engine = create_engine(connection_string, echo=True)

In [64]:
customer_df.to_sql('customers', engine, schema = 'dbo', if_exists= 'replace', index = False)
product_df.to_sql('product_usage', engine, schema = 'dbo', if_exists = 'replace', index = False)
subscription_df.to_sql('subscription', engine, schema = 'dbo', if_exists = 'replace', index = False)
support_ticket_df.to_sql('support_tickets', engine, schema = 'dbo', if_exists = 'replace', index = False)

2025-07-24 00:49:17,245 INFO sqlalchemy.engine.Engine SELECT CAST(SERVERPROPERTY('ProductVersion') AS VARCHAR)
2025-07-24 00:49:17,248 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-24 00:49:17,255 INFO sqlalchemy.engine.Engine SELECT schema_name()
2025-07-24 00:49:17,257 INFO sqlalchemy.engine.Engine [generated in 0.00246s] ()
2025-07-24 00:49:17,262 INFO sqlalchemy.engine.Engine SELECT CAST('test max support' AS NVARCHAR(max))
2025-07-24 00:49:17,264 INFO sqlalchemy.engine.Engine [generated in 0.00214s] ()
2025-07-24 00:49:17,268 INFO sqlalchemy.engine.Engine SELECT 1 FROM fn_listextendedproperty(default, default, default, default, default, default, default)
2025-07-24 00:49:17,270 INFO sqlalchemy.engine.Engine [generated in 0.00226s] ()
2025-07-24 00:49:17,284 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-24 00:49:17,297 INFO sqlalchemy.engine.Engine SELECT [INFORMATION_SCHEMA].[TABLES].[TABLE_NAME] 
FROM [INFORMATION_SCHEMA].[TABLES] 
WHERE ([INFORMATION_SCHEMA].[TABLE

102